# Construire avec les modèles Mistral

## Introduction

Cette leçon abordera :
- L’exploration des différents modèles Mistral
- La compréhension des cas d’utilisation et des scénarios pour chaque modèle
- Des exemples de code illustrent les fonctionnalités uniques de chaque modèle.


## Les modèles Mistral

Dans cette leçon, nous allons découvrir trois modèles Mistral différents : **Mistral Large**, **Mistral Small** et **Mistral Nemo**.

Chacun de ces modèles est disponible gratuitement sur le marketplace de modèles Github. Le code de ce notebook utilisera ces modèles pour exécuter le code. Voici plus d’informations sur l’utilisation des modèles Github pour [prototyper avec des modèles d’IA](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 est actuellement le modèle phare de Mistral et a été conçu pour un usage en entreprise.

Ce modèle est une amélioration par rapport au Mistral Large original en offrant :
- Une fenêtre de contexte plus grande - 128k contre 32k
- De meilleures performances sur les tâches de mathématiques et de programmation - 76,9 % de précision moyenne contre 60,4 %
- Des performances multilingues accrues - les langues incluent : anglais, français, allemand, espagnol, italien, portugais, néerlandais, russe, chinois, japonais, coréen, arabe et hindi.

Grâce à ces fonctionnalités, Mistral Large excelle dans :
- *Retrieval Augmented Generation (RAG)* - grâce à la fenêtre de contexte élargie
- *Appel de fonctions* - ce modèle propose l’appel de fonctions en natif, ce qui permet l’intégration avec des outils et API externes. Ces appels peuvent être effectués en parallèle ou de manière séquentielle, les uns après les autres.
- *Génération de code* - ce modèle est particulièrement performant pour la génération de code en Python, Java, TypeScript et C++.


Dans cet exemple, nous utilisons Mistral Large 2 pour appliquer un schéma RAG sur un document texte. La question est rédigée en coréen et porte sur les activités de l’auteur avant l’université.

Le modèle Cohere Embeddings est utilisé pour créer des embeddings du document texte ainsi que de la question. Pour cet exemple, le package Python faiss est utilisé comme base de données vectorielle.

L’invite envoyée au modèle Mistral inclut à la fois la question et les extraits récupérés qui sont similaires à la question. Le modèle fournit ensuite une réponse en langage naturel.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small est un autre modèle de la famille Mistral, classé dans la catégorie premier/entreprise. Comme son nom l’indique, ce modèle est un Small Language Model (SLM). Les avantages de Mistral Small sont les suivants :
- Économique par rapport aux LLMs Mistral comme Mistral Large et NeMo – réduction de prix de 80 %
- Faible latence – réponses plus rapides que les LLMs de Mistral
- Flexible – peut être déployé dans différents environnements avec moins de contraintes sur les ressources nécessaires.

Mistral Small est idéal pour :
- Les tâches textuelles comme la synthèse, l’analyse de sentiment et la traduction.
- Les applications nécessitant des requêtes fréquentes grâce à son rapport coût-efficacité
- Les tâches de code à faible latence comme la relecture et les suggestions de code


## Comparaison entre Mistral Small et Mistral Large

Pour observer les différences de latence entre Mistral Small et Large, exécutez les cellules ci-dessous.

Vous devriez constater une différence de temps de réponse entre 3 et 5 secondes. Notez également la longueur et le style des réponses pour le même prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Comparé aux deux autres modèles abordés dans cette leçon, Mistral NeMo est le seul modèle gratuit avec une licence Apache2.

Il est considéré comme une amélioration par rapport au précédent LLM open source de Mistral, le Mistral 7B.

Voici quelques autres caractéristiques du modèle NeMo :

- *Tokenisation plus efficace :* Ce modèle utilise le tokenizer Tekken plutôt que le tiktoken, plus couramment utilisé. Cela permet de meilleures performances sur un plus grand nombre de langues et de codes.

- *Ajustement fin :* Le modèle de base est disponible pour l’ajustement fin. Cela offre plus de flexibilité pour les cas d’usage où un ajustement fin peut être nécessaire.

- *Appel de fonctions natif* – Comme Mistral Large, ce modèle a été entraîné à l’appel de fonctions. Cela le rend unique en tant que l’un des premiers modèles open source à proposer cette fonctionnalité.


## Mistral NeMo

Comparé aux deux autres modèles abordés dans cette leçon, Mistral NeMo est le seul modèle gratuit avec une licence Apache2.

Il est considéré comme une amélioration par rapport au précédent LLM open source de Mistral, Mistral 7B.

Voici quelques autres caractéristiques du modèle NeMo :

- *Tokenisation plus efficace :* Ce modèle utilise le tokenizer Tekken au lieu du tiktoken, plus couramment utilisé. Cela permet de meilleures performances sur un plus grand nombre de langues et de codes.

- *Ajustement fin :* Le modèle de base est disponible pour l’ajustement fin. Cela offre plus de flexibilité pour les cas d’usage où un ajustement fin peut être nécessaire.

- *Appel de fonctions natif* – Comme Mistral Large, ce modèle a été entraîné à l’appel de fonctions. Cela le rend unique en tant que l’un des premiers modèles open source à proposer cette fonctionnalité.


### Comparaison des tokenizeurs

Dans cet exemple, nous allons voir comment Mistral NeMo gère la tokenisation par rapport à Mistral Large.

Les deux exemples utilisent le même prompt, mais vous devriez constater que NeMo renvoie moins de tokens que Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## L'apprentissage ne s'arrête pas ici, poursuivez votre parcours

Après avoir terminé cette leçon, consultez notre [collection d'apprentissage sur l’IA générative](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) pour continuer à approfondir vos connaissances sur l’IA générative !



---

**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction par IA [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude de la traduction, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant autorité. Pour les informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
